## EVC Charging Sessions data cleaning

These are the set of libraries used in this Analysis

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize #package for flattening json in pandas df
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#avoid warnings
import warnings
warnings.filterwarnings("ignore")
#plt style
plt.style.use('fivethirtyeight')
import datetime

user_define_function

In [2]:
def converting_seconds(x):
    return ((x.hour * 3600) + (x.minute * 60) + x.second)

Now we are reading the data.The data is in JSON format,so in belowe code its reading the json file.

In [3]:
#load json object
with open('C:/Users/admin/Desktop/rohan/projects/EV/data_set/EVC_charging_sessions/charging_sessions.json') as f:
    d = json.load(f)

In _items key in json we have all the columns of our data ,so in the belowe code we are extracting all the columns

In [4]:
#form the dirty column we are going to extract original columns    
data = json_normalize(d['_items'])
data.head(3)

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5cb28090f9af8b41d62ede72,0039,"Fri, 29 Mar 2019 06:44:57 GMT","Fri, 29 Mar 2019 07:53:31 GMT","Fri, 29 Mar 2019 07:53:27 GMT",3.977,2_39_79_377_2019-03-28 23:44:57.290195,0002,CA-325,2-39-79-377,America/Los_Angeles,000000515,"[{'WhPerMile': 250, 'kWhRequested': 10.0, 'mil..."
1,5cb28090f9af8b41d62ede73,0039,"Fri, 29 Mar 2019 07:17:34 GMT","Fri, 29 Mar 2019 08:14:11 GMT","Fri, 29 Mar 2019 07:48:10 GMT",0.844,2_39_124_22_2019-03-29 00:17:34.437344,0002,CA-312,2-39-124-22,America/Los_Angeles,None,None
2,5cb28090f9af8b41d62ede74,0039,"Fri, 29 Mar 2019 08:28:42 GMT","Fri, 29 Mar 2019 09:36:13 GMT","Fri, 29 Mar 2019 09:36:04 GMT",3.723,2_39_139_28_2019-03-29 01:28:41.779204,0002,CA-303,2-39-139-28,America/Los_Angeles,000001164,"[{'WhPerMile': 525, 'kWhRequested': 10.5, 'mil..."


We have nested columns in the above dataframe.In "userInput column" ,there are many sub columns present in the data set.so by the below code we are extracting those columns and joining to the main dataframe

In [5]:
#function for cleaning of userInputs column in data 
def give_the_dirty_column(x):
    """
    what this function does?
    -------------------------
    This function takes Pandas series ,which contain Nested json files,where in a row ,the value is the list which contain only one dictionary data structure ,in which there are some n key, value pairs
    This function extract all the values of keys of dictionary and store 's these values in a parent list,which contain multiple lists which represent each keys of the dictionary  
    
    Parameters
    -------------------------
    Number of parameter :- 1
    Type of data to be passed :- Pandas Series
    Number Default parameters :- 0
    
    returns
    -------------------------
    Returns the parent list containing multiple lists ,where each list represent values of one key in the dictionary.
    
    """
    parent_list=[]
    #creating the empty list for placing all values in respective list.
    for i in range(0,len(list(x.keys()))):
        parent_list.append([])
    row_count=0
    #iterating through all the rows in the column.
    while(row_count < x.shape[0]):
        #if the row is None then insert None inside the lists
        if(x[row_count]==None):
            for i in range(0,len(parent_list)):
                parent_list[i].append(None)
            row_count+=1
        #if the row is not None the insert the values of the dict inside the list
        else:
            parent=0
            for i in x[0][0]:
                parent_list[parent].append(x[0][0][i])
                parent+=1
            row_count+=1
    return parent_list

#calling the function 
parent_list=give_the_dirty_column(data['userInputs'])  

The above function returns the parent_list which contain lists,where lists = Number of keys present in the userInput column in each row ,These list contains the values of  keys.
now we are adding those lists to the original dataframe

In [6]:
list_count=0
#Adding extracted columns to the original data set
for i in list(data['userInputs'][0][0].keys()):
    data[i]=parent_list[list_count]
    list_count+=1
#droping the dirty column
data.drop("userInputs",axis=1,inplace=True)
data.head(3)

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5cb28090f9af8b41d62ede72,0039,"Fri, 29 Mar 2019 06:44:57 GMT","Fri, 29 Mar 2019 07:53:31 GMT","Fri, 29 Mar 2019 07:53:27 GMT",3.977,2_39_79_377_2019-03-28 23:44:57.290195,0002,CA-325,2-39-79-377,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
1,5cb28090f9af8b41d62ede73,0039,"Fri, 29 Mar 2019 07:17:34 GMT","Fri, 29 Mar 2019 08:14:11 GMT","Fri, 29 Mar 2019 07:48:10 GMT",0.844,2_39_124_22_2019-03-29 00:17:34.437344,0002,CA-312,2-39-124-22,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
2,5cb28090f9af8b41d62ede74,0039,"Fri, 29 Mar 2019 08:28:42 GMT","Fri, 29 Mar 2019 09:36:13 GMT","Fri, 29 Mar 2019 09:36:04 GMT",3.723,2_39_139_28_2019-03-29 01:28:41.779204,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"


In [7]:
data

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5cb28090f9af8b41d62ede72,0039,"Fri, 29 Mar 2019 06:44:57 GMT","Fri, 29 Mar 2019 07:53:31 GMT","Fri, 29 Mar 2019 07:53:27 GMT",3.977000,2_39_79_377_2019-03-28 23:44:57.290195,0002,CA-325,2-39-79-377,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
1,5cb28090f9af8b41d62ede73,0039,"Fri, 29 Mar 2019 07:17:34 GMT","Fri, 29 Mar 2019 08:14:11 GMT","Fri, 29 Mar 2019 07:48:10 GMT",0.844000,2_39_124_22_2019-03-29 00:17:34.437344,0002,CA-312,2-39-124-22,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
2,5cb28090f9af8b41d62ede74,0039,"Fri, 29 Mar 2019 08:28:42 GMT","Fri, 29 Mar 2019 09:36:13 GMT","Fri, 29 Mar 2019 09:36:04 GMT",3.723000,2_39_139_28_2019-03-29 01:28:41.779204,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
3,5cb28090f9af8b41d62ede75,0039,"Fri, 29 Mar 2019 09:39:36 GMT","Fri, 29 Mar 2019 20:49:08 GMT","Fri, 29 Mar 2019 11:26:08 GMT",11.454000,2_39_123_23_2019-03-29 02:39:36.360277,0002,CA-313,2-39-123-23,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
4,5cb3d172f9af8b4551261e28,0039,"Fri, 29 Mar 2019 20:34:00 GMT","Fri, 29 Mar 2019 22:06:24 GMT","Fri, 29 Mar 2019 21:51:33 GMT",3.784000,2_39_89_25_2019-03-29 13:34:00.027531,0002,CA-315,2-39-89-25,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
5,5cb3d172f9af8b4551261e29,0039,"Fri, 29 Mar 2019 20:41:31 GMT","Sat, 30 Mar 2019 02:08:36 GMT","Sat, 30 Mar 2019 00:15:41 GMT",17.826000,2_39_88_24_2019-03-29 13:41:31.037770,0002,CA-314,2-39-88-24,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
6,5cb3d172f9af8b4551261e2a,0039,"Fri, 29 Mar 2019 21:58:03 GMT","Fri, 29 Mar 2019 22:33:12 GMT","Fri, 29 Mar 2019 22:33:07 GMT",3.955000,2_39_90_26_2019-03-29 14:58:03.335682,0002,CA-316,2-39-90-26,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
7,5cb3d172f9af8b4551261e2b,0039,"Fri, 29 Mar 2019 22:26:25 GMT","Sat, 30 Mar 2019 07:37:00 GMT","Sat, 30 Mar 2019 02:01:30 GMT",12.088000,2_39_139_28_2019-03-29 15:26:25.309466,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
8,5cb3d172f9af8b4551261e2c,0039,"Fri, 29 Mar 2019 22:26:30 GMT","Sat, 30 Mar 2019 09:20:27 GMT","Sat, 30 Mar 2019 00:15:08 GMT",10.495000,2_39_79_380_2019-03-29 15:26:29.796244,0002,CA-489,2-39-79-380,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
9,5cb3d172f9af8b4551261e2d,0039,"Fri, 29 Mar 2019 22:31:51 GMT","Fri, 29 Mar 2019 23:15:31 GMT","Fri, 29 Mar 2019 23:16:22 GMT",4.745000,2_39_95_27_2019-03-29 15:31:50.751725,0002,CA-319,2-39-95-27,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"


In [8]:
print(len(data['stationID'].unique()))
print(len(data['spaceID'].unique()))
data['userID'].unique()

52
52


array([515.,  nan])

### Data cleaning

Checking the data fromat of all the columns present in our data set.

In [9]:
data.dtypes

_id                    object
clusterID              object
connectionTime         object
disconnectTime         object
doneChargingTime       object
kWhDelivered          float64
sessionID              object
siteID                 object
spaceID                object
stationID              object
timezone               object
userID                float64
WhPerMile             float64
kWhRequested          float64
milesRequested        float64
minutesAvailable      float64
modifiedAt             object
paymentRequired        object
requestedDeparture     object
dtype: object

checking for any redundant Row in the data set

In [10]:
data.duplicated().sum()

0

#### Checking for NA 's

checking the NA 's values in terms of percentage

In [11]:
(data.isna().sum()/data.shape[0])*100

_id                    0.000000
clusterID              0.000000
connectionTime         0.000000
disconnectTime         0.000000
doneChargingTime       9.258136
kWhDelivered           0.000000
sessionID              0.000000
siteID                 0.000000
spaceID                0.000000
stationID              0.000000
timezone               0.000000
userID                13.988276
WhPerMile             13.988276
kWhRequested          13.988276
milesRequested        13.988276
minutesAvailable      13.988276
modifiedAt            13.988276
paymentRequired       13.988276
requestedDeparture    13.988276
dtype: float64

#### Checking for NULL values

checking total percntage of NA 's present in the data set

In [12]:
(len(data[data.isna().any(axis=1)])/len(data))*100

21.770770163735598

The 21% of rows contain NA 's in the data set.


checking the percentage of NA 's preset in each column in the data set

In [13]:
(data.isna().sum()/data.shape[0])*100

_id                    0.000000
clusterID              0.000000
connectionTime         0.000000
disconnectTime         0.000000
doneChargingTime       9.258136
kWhDelivered           0.000000
sessionID              0.000000
siteID                 0.000000
spaceID                0.000000
stationID              0.000000
timezone               0.000000
userID                13.988276
WhPerMile             13.988276
kWhRequested          13.988276
milesRequested        13.988276
minutesAvailable      13.988276
modifiedAt            13.988276
paymentRequired       13.988276
requestedDeparture    13.988276
dtype: float64

#### Handling NA values

First let us check in the column "doneChargingTime",why 9% of rows are NaN in this column

In [14]:
data[data['doneChargingTime'].isna()==True]

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
47,5cb52257f9af8b48cb4bfe1f,0039,"Sun, 31 Mar 2019 03:16:33 GMT","Sun, 31 Mar 2019 05:02:36 GMT",None,5.346,2_39_131_30_2019-03-30 20:16:33.162954,0002,CA-305,2-39-131-30,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
278,5cbfae0df9af8b1c4fdea9eb,0039,"Sun, 07 Apr 2019 22:15:45 GMT","Sun, 07 Apr 2019 23:45:40 GMT",None,5.205,2_39_139_28_2019-04-07 15:15:45.197884,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
279,5cbfae0df9af8b1c4fdea9ec,0039,"Sun, 07 Apr 2019 23:20:28 GMT","Mon, 08 Apr 2019 02:54:11 GMT",None,6.881,2_39_79_378_2019-04-07 16:20:27.619213,0002,CA-326,2-39-79-378,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
280,5cbfae0df9af8b1c4fdea9ed,0039,"Mon, 08 Apr 2019 00:05:41 GMT","Mon, 08 Apr 2019 03:12:17 GMT",None,10.660,2_39_131_30_2019-04-07 17:05:40.978352,0002,CA-305,2-39-131-30,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
281,5cbfae0df9af8b1c4fdea9ee,0039,"Mon, 08 Apr 2019 00:19:32 GMT","Mon, 08 Apr 2019 06:29:42 GMT",None,11.646,2_39_83_386_2019-04-07 17:19:31.972571,0002,CA-212,2-39-83-386,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
282,5cbfae0df9af8b1c4fdea9ef,0039,"Mon, 08 Apr 2019 01:01:29 GMT","Mon, 08 Apr 2019 03:02:52 GMT",None,0.953,2_39_130_31_2019-04-07 18:01:29.145682,0002,CA-306,2-39-130-31,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
283,5cbfae0df9af8b1c4fdea9f0,0039,"Mon, 08 Apr 2019 02:46:12 GMT","Mon, 08 Apr 2019 04:32:12 GMT",None,5.950,2_39_79_379_2019-04-07 19:46:12.133580,0002,CA-327,2-39-79-379,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
284,5cbfae0df9af8b1c4fdea9f1,0039,"Mon, 08 Apr 2019 03:54:38 GMT","Mon, 08 Apr 2019 05:18:34 GMT",None,8.526,2_39_131_30_2019-04-07 20:54:35.903764,0002,CA-305,2-39-131-30,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
285,5cbfae0df9af8b1c4fdea9f2,0039,"Mon, 08 Apr 2019 05:18:58 GMT","Mon, 08 Apr 2019 07:29:56 GMT",None,14.642,2_39_129_17_2019-04-07 22:18:47.482199,0002,CA-307,2-39-129-17,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"
286,5cbfae0df9af8b1c4fdea9f3,0039,"Mon, 08 Apr 2019 06:37:47 GMT","Mon, 08 Apr 2019 11:17:03 GMT",None,28.512,2_39_127_19_2019-04-07 23:37:46.735878,0002,CA-309,2-39-127-19,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,"Fri, 29 Mar 2019 04:02:57 GMT"


The data of doneCharging time is not captured for these dates in these rows,

In [15]:
data[data.isna().any(axis=1)][data['userID'].isna()==True]

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
1,5cb28090f9af8b41d62ede73,0039,"Fri, 29 Mar 2019 07:17:34 GMT","Fri, 29 Mar 2019 08:14:11 GMT","Fri, 29 Mar 2019 07:48:10 GMT",0.844000,2_39_124_22_2019-03-29 00:17:34.437344,0002,CA-312,2-39-124-22,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
18,5cb3d172f9af8b4551261e36,0039,"Fri, 29 Mar 2019 23:48:44 GMT","Sat, 30 Mar 2019 02:44:13 GMT","Sat, 30 Mar 2019 02:00:13 GMT",0.848000,2_39_139_567_2019-03-29 16:48:43.958086,0002,CA-513,2-39-139-567,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
23,5cb3d172f9af8b4551261e3b,0039,"Sat, 30 Mar 2019 01:02:11 GMT","Sat, 30 Mar 2019 07:54:38 GMT","Sat, 30 Mar 2019 07:54:33 GMT",10.007000,2_39_79_381_2019-03-29 18:02:10.966435,0002,CA-490,2-39-79-381,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
27,5cb3d172f9af8b4551261e3f,0039,"Sat, 30 Mar 2019 03:30:27 GMT","Sat, 30 Mar 2019 08:35:44 GMT","Sat, 30 Mar 2019 05:41:56 GMT",0.987000,2_39_129_17_2019-03-29 20:30:27.462699,0002,CA-307,2-39-129-17,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
29,5cb3d172f9af8b4551261e41,0039,"Sat, 30 Mar 2019 03:39:29 GMT","Sat, 30 Mar 2019 06:58:25 GMT","Sat, 30 Mar 2019 06:45:05 GMT",0.901000,2_39_78_361_2019-03-29 20:39:28.908647,0002,CA-493,2-39-78-361,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
34,5cb3d172f9af8b4551261e46,0039,"Sat, 30 Mar 2019 07:12:54 GMT","Sat, 30 Mar 2019 08:20:05 GMT","Sat, 30 Mar 2019 07:44:05 GMT",0.901000,2_39_78_361_2019-03-30 00:12:53.539748,0002,CA-493,2-39-78-361,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
51,5cb52257f9af8b48cb4bfe23,0039,"Sun, 31 Mar 2019 09:54:47 GMT","Sun, 31 Mar 2019 13:48:09 GMT","Sun, 31 Mar 2019 10:25:39 GMT",0.870000,2_39_79_383_2019-03-31 02:54:47.288598,0002,CA-492,2-39-79-383,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
71,5cb7c521f9af8b50839305c5,0039,"Mon, 01 Apr 2019 23:16:03 GMT","Tue, 02 Apr 2019 06:56:36 GMT","Mon, 01 Apr 2019 23:46:20 GMT",0.837000,2_39_79_382_2019-04-01 16:16:03.463213,0002,CA-491,2-39-79-382,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
74,5cb7c521f9af8b50839305c8,0039,"Mon, 01 Apr 2019 23:21:18 GMT","Tue, 02 Apr 2019 08:33:48 GMT","Tue, 02 Apr 2019 06:23:33 GMT",0.917000,2_39_95_27_2019-04-01 16:21:18.425884,0002,CA-319,2-39-95-27,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None
83,5cb7c521f9af8b50839305d1,0039,"Tue, 02 Apr 2019 00:24:17 GMT","Tue, 02 Apr 2019 03:23:27 GMT","Tue, 02 Apr 2019 02:18:10 GMT",0.814000,2_39_125_21_2019-04-01 17:24:16.826653,0002,CA-311,2-39-125-21,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,None


The 13% of NA 's present in user inputcolumns[userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,doneChargingTime] is due to the users have not given the userInput or the data is not recorded of the userInput.  

Now we are creating two clean set data samples,where data set 1 is full clean set data in which we do not have any NA values.
In Data set2 which we are creating ,we have not droping those rows where user input has not been recorded ,we have dropped only
those rows where DoneChargingTime has not been recorded.

##### Data set 1

In [16]:
data1=data.dropna()

##### Data Set 2

In [17]:
data2 = data.dropna(subset=['doneChargingTime'])

In [18]:
data2.isna().sum()

_id                     0
clusterID               0
connectionTime          0
disconnectTime          0
doneChargingTime        0
kWhDelivered            0
sessionID               0
siteID                  0
spaceID                 0
stationID               0
timezone                0
userID                619
WhPerMile             619
kWhRequested          619
milesRequested        619
minutesAvailable      619
modifiedAt            619
paymentRequired       619
requestedDeparture    619
dtype: int64

In [19]:
print("The number of rows of data set 1 :- ",len(data1.isna()))
print("The number of rows of data set 2 :- ",len(data1.isna()))

The number of rows of data set 1 :-  3870
The number of rows of data set 2 :-  3870


#### Now we are extracting all weekdays data from "connectionTime" column and changing converting these list [connectionTime,disconnectTime ,doneChargingTime ,modifiedAt	,requestedDeparture]of column from string to date time format.

Now in the below cell,we are extracting all the weekdays and cleaning the connectingTime column.Now we are also cleaning disconnectTime , doneChargingTime ,requestedDeparture time

In [20]:
data1['Connecting_week_days']=data1['connectionTime'].str.split(',', expand=True)[0]
data1['connectionTime']=pd.to_datetime(data1['connectionTime'].str.split(',', expand=True)[1])
data1['disconnectTime']=pd.to_datetime(data1['disconnectTime'].str.split(',',expand=True)[1])
data1['doneChargingTime']=pd.to_datetime(data1['doneChargingTime'].str.split(',',expand=True)[1])
data1['requestedDeparture']=pd.to_datetime(data1['requestedDeparture'].str.split(',',expand=True)[1])

In [21]:
data2['Connecting_week_days']=data2['connectionTime'].str.split(',', expand=True)[0]
data2['connectionTime']=pd.to_datetime(data2['connectionTime'].str.split(',', expand=True)[1])
data2['disconnectTime']=pd.to_datetime(data2['disconnectTime'].str.split(',',expand=True)[1])
data2['doneChargingTime']=pd.to_datetime(data2['doneChargingTime'].str.split(',',expand=True)[1])
data2['requestedDeparture']=pd.to_datetime(data2['requestedDeparture'].str.split(',',expand=True)[1])

##### Creating the new column 
- Total_charging_time(seconds) = (disconnectTime - connectionTime)
- Time took to charge(seconds) = Done charging time of EV - Connecting charging time of EV

In [28]:
#Data set 1
data1['Total charging time']=abs(data1['disconnectTime'].dt.time.apply(converting_seconds)-data1['connectionTime'].dt.time.apply(converting_seconds))
data1['Time took to charge']=abs(data1['doneChargingTime'].dt.time.apply(converting_seconds)-data1['connectionTime'].dt.time.apply(converting_seconds))
data1['speed_transmittion(KW/sec)']=data1['kWhDelivered']/data1['Time took to charge']
#Data set 2
data2['Total charging time']=abs(data2['disconnectTime'].dt.time.apply(converting_seconds)-data2['connectionTime'].dt.time.apply(converting_seconds))
data2['Time took to charge']=abs(data2['doneChargingTime'].dt.time.apply(converting_seconds)-data2['connectionTime'].dt.time.apply(converting_seconds))
data2['speed_transmittion(KW/sec)']=data2['kWhDelivered']/data2['Time took to charge']

In [29]:
pd.set_option('display.max_columns', 500)
data2

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,Connecting_week_days,Total charging time,Time took to charge,speed_transmittion(KW/sec)
0,5cb28090f9af8b41d62ede72,0039,2019-03-29 06:44:57+00:00,2019-03-29 07:53:31+00:00,2019-03-29 07:53:27+00:00,3.977000,2_39_79_377_2019-03-28 23:44:57.290195,0002,CA-325,2-39-79-377,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,4114,4110,0.000968
1,5cb28090f9af8b41d62ede73,0039,2019-03-29 07:17:34+00:00,2019-03-29 08:14:11+00:00,2019-03-29 07:48:10+00:00,0.844000,2_39_124_22_2019-03-29 00:17:34.437344,0002,CA-312,2-39-124-22,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,None,None,NaT,Fri,3397,1836,0.000460
2,5cb28090f9af8b41d62ede74,0039,2019-03-29 08:28:42+00:00,2019-03-29 09:36:13+00:00,2019-03-29 09:36:04+00:00,3.723000,2_39_139_28_2019-03-29 01:28:41.779204,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,4051,4042,0.000921
3,5cb28090f9af8b41d62ede75,0039,2019-03-29 09:39:36+00:00,2019-03-29 20:49:08+00:00,2019-03-29 11:26:08+00:00,11.454000,2_39_123_23_2019-03-29 02:39:36.360277,0002,CA-313,2-39-123-23,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,40172,6392,0.001792
4,5cb3d172f9af8b4551261e28,0039,2019-03-29 20:34:00+00:00,2019-03-29 22:06:24+00:00,2019-03-29 21:51:33+00:00,3.784000,2_39_89_25_2019-03-29 13:34:00.027531,0002,CA-315,2-39-89-25,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,5544,4653,0.000813
5,5cb3d172f9af8b4551261e29,0039,2019-03-29 20:41:31+00:00,2019-03-30 02:08:36+00:00,2019-03-30 00:15:41+00:00,17.826000,2_39_88_24_2019-03-29 13:41:31.037770,0002,CA-314,2-39-88-24,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,66775,73550,0.000242
6,5cb3d172f9af8b4551261e2a,0039,2019-03-29 21:58:03+00:00,2019-03-29 22:33:12+00:00,2019-03-29 22:33:07+00:00,3.955000,2_39_90_26_2019-03-29 14:58:03.335682,0002,CA-316,2-39-90-26,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,2109,2104,0.001880
7,5cb3d172f9af8b4551261e2b,0039,2019-03-29 22:26:25+00:00,2019-03-30 07:37:00+00:00,2019-03-30 02:01:30+00:00,12.088000,2_39_139_28_2019-03-29 15:26:25.309466,0002,CA-303,2-39-139-28,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,53365,73495,0.000164
8,5cb3d172f9af8b4551261e2c,0039,2019-03-29 22:26:30+00:00,2019-03-30 09:20:27+00:00,2019-03-30 00:15:08+00:00,10.495000,2_39_79_380_2019-03-29 15:26:29.796244,0002,CA-489,2-39-79-380,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,47163,79882,0.000131
9,5cb3d172f9af8b4551261e2d,0039,2019-03-29 22:31:51+00:00,2019-03-29 23:15:31+00:00,2019-03-29 23:16:22+00:00,4.745000,2_39_95_27_2019-03-29 15:31:50.751725,0002,CA-319,2-39-95-27,America/Los_Angeles,515.0,250.0,10.0,40.0,258.0,"Thu, 28 Mar 2019 23:46:06 GMT",True,2019-03-29 04:02:57+00:00,Fri,2620,2671,0.001776


In [25]:
3.977000/4110

0.000967639902676399

In [26]:
4110*0.000967639902676399

3.977

Saving the clean data into CSV

In [42]:
data1.to_csv("session_data_clean.csv")
data2.to_csv("session_data_clean_partially.csv")